## Import some 'braries

In [1]:
import pandas as pd
pd.set_option('display.max_columns',100)

# from scipy.spatial.distance import euclidean as euc
import numpy as np
np.random.seed(0)

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

## Original

### Setting up df

# Creating initial dataframe
df1 = pd.read_pickle('final_df.pickle')

dupes = df1[df1.duplicated(['pid','age'],keep=False)].sort_values(by=['player','g'],ascending=False)

df1 = df1.drop(dupes.index)

df1 = pd.concat([df1, dupes[dupes.tm == 'TOT']])

df1 = df1.rename(columns={'3p':'fg3',
                   '3pa':'fg3a',
                   '2p':'fg2',
                   '2pa':'fg2a',
                   '3p_pct':'fg3_pct',
                   '2p_pct':'fg2_pct'})

# Features of interest 
cols = ['player', 'pos', 'mp', 'fg2_pct', 'fg3_pct', 'ft_pct', 'pts', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'ws']

# Setting up df to have only recent players 
test = df1[df1.season == 2019][cols]
test.player = [name.title() for name in test.player]

# Created dummies 
test['PG']=list(map(lambda x: 1 if 'PG' in x else 0, list(test.pos)))
test['SG']=list(map(lambda x: 1 if 'SG' in x else 0, list(test.pos)))
test['SF']=list(map(lambda x: 1 if 'SF' in x else 0, list(test.pos)))
test['PF']=list(map(lambda x: 1 if 'PF' in x else 0, list(test.pos)))
test['C']=list(map(lambda x: 1 if 'C' in x else 0, list(test.pos)))

### With position specified, all stats and win-share only

position = 'PG'

player = test[test.player == input('Please enter player name: \n').title()].index[0]
player = test.loc[player]

indie = (test[test.columns[2:]] - player[2:]).abs().sum(axis=1).sort_values().index[1:]

###### Filtered by position

filt = test.loc[indie]
filt[filt.pos == position].iloc[:10]

###### Not filtered by position

filt.iloc[:10]

## Using different df

df = pd.read_pickle('stats_and_lr_preds.pickle')

### With all stats and win-share only

main = df[df.season==2019].drop(['season','season_max','draft_year','lasso_preds','ridge_preds',
                                 'lin_preds','age', 'experience','g'],axis=1)
main.player = [name.title() for name in main.player]

print('Possible positions: \n PG \n SG \n SF \n PF \n C')

unavail = main[(main.player == input('Please input player name: \n').title())].index[0]
unavail = main.loc[unavail]

ind = (main[main.columns[1:]] - unavail[1:]).abs().sum(axis=1).sort_values().index[1:11]

main.loc[ind]

### With all stats and win-share and ridge regression predicted win-share

main1 = df[df.season==2019].drop(['season','season_max','draft_year','lasso_preds',
                                 'lin_preds','age', 'experience','g'],axis=1)
main1.player = [name.title() for name in main1.player]

print('Possible positions: \n PG \n SG \n SF \n PF \n C')

unavail = main1[(main1.player == input('Please input player name: \n').title())].index[0]
unavail = main1.loc[unavail]

ind = (main1[main1.columns[1:]] - unavail[1:]).abs().sum(axis=1).sort_values().index[1:11]

main1.loc[ind]

### With all stats and ridge regression predicted win-share only

main2 = df[df.season==2019].drop(['season','season_max','draft_year','lasso_preds','ws',
                                 'lin_preds','age', 'experience','g'],axis=1)
main2.player = [name.title() for name in main2.player]

print('Possible positions: \n PG \n SG \n SF \n PF \n C')

unavail = main2[(main2.player == input('Please input player name: \n').title())].index[0]
unavail = main2.loc[unavail]

ind = (main2[main2.columns[1:]] - unavail[1:]).abs().sum(axis=1).sort_values().index[1:11]
main2.loc[ind]
# main2.merge(ind, left_index=True, right_index=True)

main2.loc[ind]

Notice no difference in recommendation list

## Function

# DataFrame
data = pd.read_pickle('final_df.pickle')

dupes = data[data.duplicated(['pid','age'],keep=False)].sort_values(by=['player','g'],ascending=False)

data = data.drop(dupes.index)

data = pd.concat([data, dupes[dupes.tm == 'TOT']])

data = data.rename(columns={'3p':'fg3',
                   '3pa':'fg3a',
                   '2p':'fg2',
                   '2pa':'fg2a',
                   '3p_pct':'fg3_pct',
                   '2p_pct':'fg2_pct'})

# Features of interest 
cols = ['player', 'pos', 'mp', 'fg2_pct', 'fg3_pct', 'ft_pct',
        'pts', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'ws']

# Setting up df to have only recent players 
main_df = data[data.season == 2019][cols]
main_df.player = [name.title() for name in main_df.player]

name = 'james harden'.title()
position = 'pg'.upper()
category = ['trb']

playa = data[data.player == name].index[0]
playa = data.loc[playa]



In [186]:
# helper functions for rec 
def check_categories(cat_list, features_list):
    cat_dict = {'1': 'pts',
                '2': 'trb',
                '3': 'ast',
                '4': 'stl',
                '5': 'blk',
                '6': 'tov',
                '7': 'fg_pct',
                '8': 'fg2_pct',
                '9': 'fg3_pct',
               '10': 'ft_pct',
               '11': 'ws'}
    feats = []
    for cat in cat_list:
        cat = cat.strip()
        try:
            while int(cat) > 11:
                cat = input(f'Category {cat} does not exist. \nPlease select category:\n')
            feats.append(cat_dict[cat])
        except ValueError:
            if cat == 'reb':
                feats.append('trb')
            elif cat == '':
                continue
            elif cat not in features_list:
                while cat not in features_list:
                    cat = input(f'Category {cat} does not exist. \nPlease replace with a valid category:\n')
                    try:
                        cat = cat_dict[cat]
                    except:
                        pass
                feats.append(cat)
            else:
                feats.append(cat)
    return feats

def check_position(pos):
    pos_dict = {'1': 'PG',
               '2': 'SG',
               '3': 'SF',
               '4': 'PF',
               '5': 'C'}
    
    return pos_dict[pos]

In [224]:
# Recommendation system 
def rec_similar_players(given_df = pd.read_pickle('final_df.pickle')):
    import pandas as pd
    
    # Import projected win shares df
#     projections = pd.read_pickle('win_share_projections.pickle')
    
    # DataFrame setup
    data = given_df
    
    if type(given_df) != pd.core.frame.DataFrame:
        data = input('Given data is not a pandas dataframe. \nPlease provide a pandas dataframe: \n')
    
    data.columns = [col.lower() for col in data.columns]
    
    dupes = data[data.duplicated(['pid','age'],keep=False)].sort_values(by=['player','g'],ascending=False)

    data = data.drop(dupes.index)

    data = pd.concat([data, dupes[dupes.tm == 'TOT']])
    
    try:
        data = data.rename(columns={'3p':'fg3',
                                    '3pa':'fg3a',
                                    '2p':'fg2',
                                    '2pa':'fg2a',
                                    '3p_pct':'fg3_pct',
                                    '2p_pct':'fg2_pct'})
    except:
        pass
    
    # Merge data with win share projections
#     data = data.merge(projections[['pid','season','projected_ws']], on=['pid','season'], how='left')
    
    # Features of interest 
    features = ['player', 'pos', 'mp', 'fg_pct', 'fg2_pct', 'fg3_pct', 'ft_pct',
                'pts', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'ws']
    
    print('Possible categories: \n 1. Points (pts) \n 2. Rebounds (trb) \n 3. Assists (ast) \n 4. Steals (stl) \n 5. Blocks (blk) \n 6. Turnovers (tov) \n 7. Field goal percentage (fg_pct) \n 8. Mid-range Field goal percentage (fg2_pct) \n 9. 3-point percentage (fg3_pct) \n 10: Free-throw percentage (ft_pct) \n 11: Win Share (ws)')
    
    # ask for categories
    categories = input('\n(Optional) \nPlease select which categories to compare (Separated by commas(,)): \n').split(',')
    
    # check if categories were given
    if categories == ['']:
        cols = features
    else:
        cols = ['player', 'pos']
        categories = check_categories(categories, features)
        categories = list(set(categories))
        cols.extend(categories)
        
    # Setting up df to have only recent players 
    main_df = data[data.season == 2019][cols]
    main_df.player = [name.title() for name in main_df.player]
    
    # ask for player name that you want to compare
    name = input('Please input player name: \n').title()
    print('')
    # check to make sure a name is given
    while name == '':
        name = input('You did not specify player. \nPlease input player name: \n').title()
        print('')
    
    # check to see if player exists in dataframe
    if name not in list(main_df.player):
        while name not in list(main_df.player):
            name = input('Player is not in current database. \nPlease input player name: \n').title()
            print('')
    
    print('Possible positions: \n 1. PG (Point Guard) \n 2. SG (Shooting Guard)\n 3. SF (Small Forward)\n 4. PF (Power Forward)\n 5. C  (Center)')
    
    # ask for a position to recommend
    position = input('\n(Optional) \nPlease specify position: \n')
    
    # check if position input is numerical
    try:
        if int(position):
            while int(position) > 5:
                position = input('Please specify a position: \n')
                
            position = check_position(position)
    except ValueError:
        pass
    
    # set player to the inputted name
    player = main_df[main_df.player == name].index[0]
    player = main_df.loc[player]
    
    # subtract all other players' stats from player and aggregate for a 'distance' from input player
    # sort resulting dataframe by this distance
    # save the indeces of these players
    indeces = (main_df[main_df.columns[2:]] - player[2:]).abs().sum(axis=1).sort_values().index
    
    # select the players from the main dataframe
    neighbors = main_df.loc[indeces].dropna(subset=cols[2:])
    
    # return the player and the top 10 neighbors
    if position == '':
#         print(neighbors.iloc[:11].reset_index().drop('index',axis=1))
        return neighbors.iloc[:11].reset_index().drop('index',axis=1)
    else:
#         print(neighbors[neighbors.pos == position.upper()].iloc[:11].reset_index().drop('index',axis=1))
        return neighbors[neighbors.pos == position.upper()].iloc[:11].reset_index().drop('index',axis=1)

In [237]:
neighbors = rec_similar_players()
neighbors

Possible categories: 
 1. Points (pts) 
 2. Rebounds (trb) 
 3. Assists (ast) 
 4. Steals (stl) 
 5. Blocks (blk) 
 6. Turnovers (tov) 
 7. Field goal percentage (fg_pct) 
 8. Mid-range Field goal percentage (fg2_pct) 
 9. 3-point percentage (fg3_pct) 
 10: Free-throw percentage (ft_pct) 
 11: Win Share (ws)

(Optional) 
Please select which categories to compare (Separated by commas(,)): 

Please input player name: 
james harden

Possible positions: 
 1. PG (Point Guard) 
 2. SG (Shooting Guard)
 3. SF (Small Forward)
 4. PF (Power Forward)
 5. C  (Center)

(Optional) 
Please specify position: 



,player,pos,mp,fg_pct,fg2_pct,fg3_pct,ft_pct,pts,trb,ast,stl,blk,tov,pf,ws
0,James Harden,PG,2867,0.442,0.528,0.368,0.879,2818,518,586,158,58,387,244,15.2
1,Paul George,SF,2841,0.438,0.484,0.386,0.839,2159,628,318,170,34,205,214,11.9
2,Damian Lillard,PG,2838,0.444,0.499,0.369,0.912,2067,371,551,88,34,212,148,12.1
3,Bradley Beal,SG,3028,0.475,0.548,0.351,0.808,2099,411,448,121,58,224,226,7.6
4,Kemba Walker,PG,2863,0.434,0.494,0.356,0.844,2102,361,484,102,34,211,131,7.4
5,Kevin Durant,SF,2702,0.521,0.587,0.353,0.885,2027,497,457,58,84,225,155,11.5
6,Blake Griffin,PF,2622,0.462,0.525,0.362,0.753,1841,565,402,52,28,253,199,8.0
7,Demar Derozan,SG,2688,0.481,0.492,0.156,0.830,1635,462,475,86,36,199,177,6.3
8,Russell Westbrook,PG,2630,0.428,0.481,0.290,0.656,1675,807,784,142,33,325,245,6.8
9,Donovan Mitchell,SG,2598,0.432,0.468,0.362,0.806,1829,316,322,106,31,218,208,5.0


In [235]:
projections = pd.read_pickle('win_share_projections.pickle')
neighbors.merge(projections[projections.season==2019][['player','projected_ws']], on = ['player'])

,player,pos,mp,fg_pct,fg2_pct,fg3_pct,ft_pct,pts,trb,ast,stl,blk,tov,pf,ws,projected_ws
0,James Harden,PG,2867,0.442,0.528,0.368,0.879,2818,518,586,158,58,387,244,15.2,27.628349
1,Paul George,SF,2841,0.438,0.484,0.386,0.839,2159,628,318,170,34,205,214,11.9,21.253915
2,Damian Lillard,PG,2838,0.444,0.499,0.369,0.912,2067,371,551,88,34,212,148,12.1,20.163179
3,Bradley Beal,SG,3028,0.475,0.548,0.351,0.808,2099,411,448,121,58,224,226,7.6,18.169216
4,Kemba Walker,PG,2863,0.434,0.494,0.356,0.844,2102,361,484,102,34,211,131,7.4,17.155297
5,Kevin Durant,SF,2702,0.521,0.587,0.353,0.885,2027,497,457,58,84,225,155,11.5,21.049306
6,Blake Griffin,PF,2622,0.462,0.525,0.362,0.753,1841,565,402,52,28,253,199,8.0,15.841684
7,Russell Westbrook,PG,2630,0.428,0.481,0.290,0.656,1675,807,784,142,33,325,245,6.8,16.255561
8,Donovan Mitchell,SG,2598,0.432,0.468,0.362,0.806,1829,316,322,106,31,218,208,5.0,11.514747
9,Ben Simmons,PG,2700,0.563,0.566,0.000,0.600,1337,697,610,112,61,274,209,8.2,16.747472
